# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 21 2023 10:17AM,259622,18,WHC-660271,WHC Labs bvba,Released
1,Mar 21 2023 10:16AM,259621,10,MSP219619,"Methapharm, Inc.",Released
2,Mar 21 2023 10:16AM,259620,19,9287201,"Snap Medical Industries, LLC",Released
3,Mar 21 2023 10:05AM,259475,15,30014848,"Alliance Pharma, Inc.",Released
4,Mar 21 2023 10:05AM,259475,15,30014849,"Alliance Pharma, Inc.",Released
5,Mar 21 2023 10:05AM,259475,15,30014850,"Alliance Pharma, Inc.",Released
6,Mar 21 2023 10:05AM,259475,15,30014851,"Alliance Pharma, Inc.",Released
7,Mar 21 2023 10:05AM,259475,15,30014852,"Alliance Pharma, Inc.",Released
8,Mar 21 2023 10:05AM,259475,15,30014853,"Alliance Pharma, Inc.",Released
9,Mar 21 2023 10:05AM,259475,15,30014865,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,259618,Released,1
21,259619,Released,1
22,259620,Released,1
23,259621,Released,1
24,259622,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
259618,NaN,1.0
259619,NaN,1.0
259620,NaN,1.0
259621,NaN,1.0
259622,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
259475,0.0,20.0
259574,1.0,0.0
259596,1.0,0.0
259597,0.0,1.0
259599,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
259475,0,20
259574,1,0
259596,1,0
259597,0,1
259599,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,259475,0,20
1,259574,1,0
2,259596,1,0
3,259597,0,1
4,259599,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,259475,,20
1,259574,1,
2,259596,1,
3,259597,,1
4,259599,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 21 2023 10:17AM,259622,18,WHC Labs bvba
1,Mar 21 2023 10:16AM,259621,10,"Methapharm, Inc."
2,Mar 21 2023 10:16AM,259620,19,"Snap Medical Industries, LLC"
3,Mar 21 2023 10:05AM,259475,15,"Alliance Pharma, Inc."
23,Mar 21 2023 10:03AM,259619,18,"Emersa Waterbox, LLC"
24,Mar 21 2023 10:00AM,259618,18,"Emersa Waterbox, LLC"
25,Mar 21 2023 9:59AM,259617,18,"Emersa Waterbox, LLC"
26,Mar 21 2023 9:56AM,259616,18,"Emersa Waterbox, LLC"
27,Mar 21 2023 9:37AM,259615,10,Hush Hush
28,Mar 21 2023 9:21AM,259614,18,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 21 2023 10:17AM,259622,18,WHC Labs bvba,,1
1,Mar 21 2023 10:16AM,259621,10,"Methapharm, Inc.",,1
2,Mar 21 2023 10:16AM,259620,19,"Snap Medical Industries, LLC",,1
3,Mar 21 2023 10:05AM,259475,15,"Alliance Pharma, Inc.",,20
4,Mar 21 2023 10:03AM,259619,18,"Emersa Waterbox, LLC",,1
5,Mar 21 2023 10:00AM,259618,18,"Emersa Waterbox, LLC",,1
6,Mar 21 2023 9:59AM,259617,18,"Emersa Waterbox, LLC",,1
7,Mar 21 2023 9:56AM,259616,18,"Emersa Waterbox, LLC",,1
8,Mar 21 2023 9:37AM,259615,10,Hush Hush,,1
9,Mar 21 2023 9:21AM,259614,18,"Emersa Waterbox, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 21 2023 10:17AM,259622,18,WHC Labs bvba,1,
1,Mar 21 2023 10:16AM,259621,10,"Methapharm, Inc.",1,
2,Mar 21 2023 10:16AM,259620,19,"Snap Medical Industries, LLC",1,
3,Mar 21 2023 10:05AM,259475,15,"Alliance Pharma, Inc.",20,
4,Mar 21 2023 10:03AM,259619,18,"Emersa Waterbox, LLC",1,
5,Mar 21 2023 10:00AM,259618,18,"Emersa Waterbox, LLC",1,
6,Mar 21 2023 9:59AM,259617,18,"Emersa Waterbox, LLC",1,
7,Mar 21 2023 9:56AM,259616,18,"Emersa Waterbox, LLC",1,
8,Mar 21 2023 9:37AM,259615,10,Hush Hush,1,
9,Mar 21 2023 9:21AM,259614,18,"Emersa Waterbox, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 21 2023 10:17AM,259622,18,WHC Labs bvba,1,
1,Mar 21 2023 10:16AM,259621,10,"Methapharm, Inc.",1,
2,Mar 21 2023 10:16AM,259620,19,"Snap Medical Industries, LLC",1,
3,Mar 21 2023 10:05AM,259475,15,"Alliance Pharma, Inc.",20,
4,Mar 21 2023 10:03AM,259619,18,"Emersa Waterbox, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 21 2023 10:17AM,259622,18,WHC Labs bvba,1.0,NaN
1,Mar 21 2023 10:16AM,259621,10,"Methapharm, Inc.",1.0,NaN
2,Mar 21 2023 10:16AM,259620,19,"Snap Medical Industries, LLC",1.0,NaN
3,Mar 21 2023 10:05AM,259475,15,"Alliance Pharma, Inc.",20.0,NaN
4,Mar 21 2023 10:03AM,259619,18,"Emersa Waterbox, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 21 2023 10:17AM,259622,18,WHC Labs bvba,1.0,0.0
1,Mar 21 2023 10:16AM,259621,10,"Methapharm, Inc.",1.0,0.0
2,Mar 21 2023 10:16AM,259620,19,"Snap Medical Industries, LLC",1.0,0.0
3,Mar 21 2023 10:05AM,259475,15,"Alliance Pharma, Inc.",20.0,0.0
4,Mar 21 2023 10:03AM,259619,18,"Emersa Waterbox, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2596047,17.0,18.0
15,259475,20.0,0.0
18,1817315,7.0,0.0
19,1038427,45.0,32.0
20,259597,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2596047,17.0,18.0
1,15,259475,20.0,0.0
2,18,1817315,7.0,0.0
3,19,1038427,45.0,32.0
4,20,259597,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,17.0,18.0
1,15,20.0,0.0
2,18,7.0,0.0
3,19,45.0,32.0
4,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,17.0
1,15,Released,20.0
2,18,Released,7.0
3,19,Released,45.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,20
Status,,,,,
Executing,18.0,0.0,0.0,32.0,0.0
Released,17.0,20.0,7.0,45.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,20
0,Executing,18.0,0.0,0.0,32.0,0.0
1,Released,17.0,20.0,7.0,45.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,20
0,Executing,18.0,0.0,0.0,32.0,0.0
1,Released,17.0,20.0,7.0,45.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()